In [20]:
import geemap
import ee
# ee.Authenticate()
# ee.Initialize()

In [21]:
# ee.Authenticate()

In [22]:
ee.Initialize()

In [23]:
# Import the shapefile

# poly = ee.FeatureCollection("projects/ee-wang25/assets/Algonquin_Boundary").geometry()

In [24]:
# These are the vertices for a random selected location northwest of Noell Lake

vertices = [
    [
       -78.68027543618818,
        45.68475395194633
    ],
    [
       -78.67405271127363,
        45.68475395194633 
    ],
    [
        -78.67405271127363,
        45.68904105696354
    ],
    [
        -78.68027543618818,
        45.68904105696354
    ],
    [
        -78.68027543618818,
        45.68475395194633
    ]
]

poly = ee.Geometry.Polygon(vertices)

In [25]:
def getBit(n):
    # Returns a GEE server-side object representing `int(2^n)`
    return ee.Number(2).pow(n).int()

def addMaskBand(image):   
    qa = image.select("QA_PIXEL")
    
    dilatedCloudBit = getBit(1)
    cirrusBit = getBit(2)
    cloudBit = getBit(3)
    cloudShadowBit = getBit(4)
    snowBit = getBit(5)
    
    # Define the mask by extracting these bits and reclassifying the pixel based on the bit's value
    mask = ee.Image(0)\
        .where(qa.bitwiseAnd(dilatedCloudBit).neq(0), 1)\
        .where(qa.bitwiseAnd(cloudBit).neq(0), 2)\
        .where(qa.bitwiseAnd(cirrusBit).neq(0), 3)\
        .where(qa.bitwiseAnd(cloudShadowBit).neq(0), 4)\
        .where(qa.bitwiseAnd(snowBit).neq(0), 5)\
        .updateMask(image.select('QA_PIXEL').mask())\
        .rename("cloud_shadow_snow_mask")
    
    # return original image with this mask added as an extra band
    return image.addBands(mask)

def maskImage(image):
    cloud_shadow_snow = image.select("cloud_shadow_snow_mask")
    return image.updateMask(cloud_shadow_snow.eq(0))

def calcMNDWIL8(image):
    return image.normalizedDifference(['SR_B3', 'SR_B6']).rename("MNDWI").multiply(10000).set({'system:time_start': image.get('system:time_start')})

def calcMNDWI(image):
    return image.normalizedDifference(['SR_B2', 'SR_B5']).rename("MNDWI").multiply(10000).set({'system:time_start': image.get('system:time_start')})

In [26]:
filters = [
    ee.Filter.bounds(poly),
    ee.Filter.lt("CLOUD_COVER", 50)
]

l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filter(filters).map(addMaskBand).map(maskImage).map(calcMNDWIL8)
l7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2").filter(filters).map(addMaskBand).map(maskImage).map(calcMNDWI)
l5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2").filter(filters).map(addMaskBand).map(maskImage).map(calcMNDWI)

mndwi = ee.ImageCollection(l5.merge(l7).merge(l8)).sort('system:time_start')

In [27]:
img = mndwi.toBands().int()

In [28]:
### just in case the band names are not kept when the image is exported
### save these ones to rename them offline
bandNames = img.bandNames().getInfo()

with open('band_names_algonquin_test.txt', 'w') as file:
    for b in bandNames:
        file.write(f"{b}\n")

In [29]:
from ee.batch import Export

task = Export.image.toDrive(
    image = img,
    region = poly,
    scale = 30,
    folder = 'gee_exports',
    fileNamePrefix = "MNDWI_Algonquin_test",
    description = "Export MNDWI time series Algonquin test",
    crs = "EPSG:32617"
)

task.start()